In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats
import statsmodels.formula.api as smf
from sklearn import linear_model
from google.colab import files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
L2Mc = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/L2M cleaned.csv')

In [4]:
# 5320 observations, 36 variables
L2Mc.shape

(5320, 36)

In [5]:
L2Mc.head(5)

,Unnamed: 0,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,...,type2,season,playoff,Favored Team,FT_is_ST,Num_SP_in_FT,non Favored Team,Num_SP_in_NFT,NFT_is_ST,ST_in_call
0,0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,TRAVELING,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
1,1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,OUT OF BOUNDS,2015,False,BOS,0,0.0,GSW,1.0,1.0,1.0
2,2,Q4,Dante Cunningham,Wilson Chandler,INC,"Cunningham (NOP) clamps Chandler's (DEN) arm, ...","Pelicans @ Nuggets (Mar 01, 2015)",1-Mar-15,NOP,DEN,...,LOOSE BALL,2015,False,NOP,0,0.0,DEN,0.0,0.0,0.0
3,3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,DOUBLE PERSONAL,2015,False,CLE,1,2.0,HOU,0.0,0.0,1.0
4,4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,OFFENSIVE,2015,False,HOU,0,0.0,CLE,2.0,1.0,1.0


In [6]:
# How many favored call for teams has 0/1/2/3 super player(at least one side)

no_SP_fc = L2Mc.loc[L2Mc['Num_SP_in_FT'] == 0, 'Num_SP_in_FT'].count()
one_SP_fc = L2Mc.loc[L2Mc['Num_SP_in_FT'] == 1, 'Num_SP_in_FT'].count()
two_SP_fc = L2Mc.loc[L2Mc['Num_SP_in_FT'] == 2, 'Num_SP_in_FT'].count()
three_SP_fc = L2Mc.loc[L2Mc['Num_SP_in_FT'] == 3, 'Num_SP_in_FT'].count()

print(no_SP_fc)
print(one_SP_fc)
print(two_SP_fc)
print(three_SP_fc)

4014
949
292
65


In [7]:
# Second EDA Graph

# 202 teams have 0 super player
# 51 teams have 1 super players
# 14 teams have 2 super players
# 3 teams have 3 super players

# average call for teams with different number of super players
print(4014/202)
print(949/51)
print(292/14)
print(65/3)

19.871287128712872
18.607843137254903
20.857142857142858
21.666666666666668


## For games that only one side has super player

In [8]:
# Find all the games that at least one side has super player
L2Mc1 = L2Mc.loc[L2Mc['ST_in_call']==1]
L2Mc1.shape

(1835, 36)

In [9]:
# number of favored called to Super Team(only one side has super player) (0/1)
L2Mc1['FT_is_ST'].sum()

897

In [10]:
# First EDA data

L2Mc1['FT_is_ST'].sum()/len(L2Mc1)

0.4888283378746594

In [15]:
Super_player = pd.read_csv('/content/drive/Shared drives/DSO 510 Final Project/Super players.csv')

## For games that at least one side have super players

In [20]:
# Find all the games that at least one side has super player
L2Mc3 = L2Mc.loc[L2Mc['ST_in_call']!=0]
L2Mc3.shape

(2244, 36)

In [21]:
# Number of different type of favored calls in Super Team (at least one side is super team)

INC_FT_ST= L2Mc3.loc[(L2Mc3['decision'] == 'INC') & (L2Mc3['FT_is_ST'] == 1), 'Favored Team'].count()
IC_FT_ST= L2Mc3.loc[(L2Mc3['decision'] == 'IC') & (L2Mc3['FT_is_ST'] == 1), 'Favored Team'].count()
print(INC_FT_ST)
print(IC_FT_ST)

1166
140


In [22]:
L2Mc3.loc[(L2Mc3['decision'] == 'INC') & (L2Mc3['FT_is_ST'] == 1), 'Favored Team'].count()/(1186+131)

0.8853454821564161

In [23]:
L2Mc3.loc[(L2Mc3['decision'] == 'IC') & (L2Mc3['FT_is_ST'] == 1), 'Favored Team'].count()/(1186+131)

0.10630220197418375

In [24]:
L2Mc3.loc[L2Mc3['decision']== 'INC','Favored Player'] = L2Mc3.loc[L2Mc3['decision']== 'INC', 'committing']
L2Mc3.loc[L2Mc3['decision']== 'IC','Favored Player'] = L2Mc3.loc[L2Mc3['decision']== 'IC', 'disadvantaged']

<ipython-input-24-9911135f13c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Mc3.loc[L2Mc3['decision']== 'INC','Favored Player'] = L2Mc3.loc[L2Mc3['decision']== 'INC', 'committing']


In [25]:
merged2 = L2Mc3.merge(Super_player, left_on=['season', 'Favored Team'], right_on=['Year', 'Team'], how='left')
merged2.head(5)

,Unnamed: 0,period,committing,disadvantaged,decision,comments,game_details,game_date,away_team,home_team,...,Num_SP_in_NFT,NFT_is_ST,ST_in_call,Favored Player,Team,Player1,Player2,Player3,Counts,Year
0,0,Q4,Jae Crowder,Harrison Barnes,INC,Crowder (BOS) moves his left pivot foot after ...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,1.0,1.0,1.0,Jae Crowder,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Q4,Avery Bradley,NaN,INC,Bradley (BOS) steps out of bounds while pump f...,"Warriors @ Celtics (Mar 01, 2015)",1-Mar-15,GSW,BOS,...,1.0,1.0,1.0,Avery Bradley,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Q5,LeBron James,James Harden,INC,Harden (HOU) walks into space occupied by Jame...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,0.0,0.0,1.0,LeBron James,CLE,LeBron James,Kyrie Irving,NaN,2.0,2015.0
3,4,Q5,James Harden,LeBron James,INC,As Harden (HOU) starts his drive to the basket...,"Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,2.0,1.0,1.0,James Harden,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Q5,LeBron James,James Harden,INC,"After the initial hook by Harden (HOU), James ...","Cavaliers @ Rockets (Mar 01, 2015)",1-Mar-15,CLE,HOU,...,0.0,0.0,1.0,LeBron James,CLE,LeBron James,Kyrie Irving,NaN,2.0,2015.0


In [26]:
# 641 players in 9 season

# probability of super player
print(26/641)

# probability of non-super player
print(615/641)

0.0405616224648986
0.9594383775351014


In [27]:
# Third EDA Data
# For Favored called to Super Team, probability of favored call to Super Player

merged2.loc[(merged2['Favored Player'] == merged2['Player1'])|(merged2['Favored Player'] == merged2['Player2'])|(merged2['Favored Player'] == merged2['Player3']), 'Favored Player'].count()/len(merged2)

0.14037433155080214